In [10]:
from __future__ import absolute_import, division, print_function
import matplotlib as plt
from matplotlib.pyplot import imshow
import numpy as np
from random import sample
import tensorflow.keras.datasets as datasets
from dnn_tf import dnn
import tensorflow as tf
import math
import matplotlib.pyplot as plt
import time
# import tensorflow as tf
import numpy as np
%matplotlib inline
tf.logging.set_verbosity(tf.logging.INFO)

In [75]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
#   tf.print(logits.eval())
#   print(logits.shape)
#   print(labels.shape)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.reduce_mean(tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=tf.clip_by_value(logits, 1e-10, 1.0)))

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [76]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()
train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

In [77]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=int32)

In [78]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f72e655c908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [79]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

In [80]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=y_train_fuzzy,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1006
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1006 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.11226302 0.0613554  0.08376507 0.15472358 0.07867522 0.12092423
  0.08031538 0.08294524 0.12572901 0.09930388]
 [0.24786286 0.05073787 0.11380042 0.10087833 0.06002444 0.09268016
  0.07978954 0.06601145 0.09955756 0.08865741]
 [0.12433188 0.0717249  0.08270463 0.10290583 0.11995097 0.10941685
  0.08141026 0.09393602 0.09227473 0.12134399]
 [0.11712205 0.0530484  0.12706028 0.10680802 0.13832471 0.09095147
  0.13160829 0.06379281 0.09875138 0.07253248]
 [0.14850071 0.05113525 0.08158874 0.153638   0.08194692 0.08768038
  0.09815788 0.09516415 0.13322635

INFO:tensorflow:loss = 11.51365, step = 1007
INFO:tensorflow:Saving checkpoints for 1007 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 11.51365.


In [81]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1007
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1007 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 11.87251, step = 1008
INFO:tensorflow:global_step/sec: 5.00398
INFO:tensorflow:loss = 11.6729765, step = 1108 (19.987 sec)
INFO:tensorflow:global_step/sec: 5.01169
INFO:tensorflow:loss = 12.08838, step = 1208 (19.951 sec)
INFO:tensorflow:global_step/sec: 5.06045
INFO:tensorflow:loss = 11.775213, step = 1308 (19.761 sec)


KeyboardInterrupt: 

In [9]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-12T14:55:29Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-12-14:55:36
INFO:tensorflow:Saving dict for global step 1001: accuracy = 0.669, global_step = 1001, loss = 1.9816879
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1001: /tmp/mnist_convnet_model/model.ckpt-1001
{'accuracy': 0.669, 'loss': 1.9816879, 'global_step': 1001}


In [13]:
y_train_fuzzy = np.load("y_train_fuzzy.npy")

In [41]:
y_train_fuzzy[1]

array([9.34812623e-01, 3.06922108e-02, 7.70561590e-01, 7.05911284e-01,
       6.17975046e-01, 7.69332321e-01, 5.69919431e-01, 2.49260186e-05,
       7.57292339e-01, 3.92808006e-01])